## 1、可视化DataGeneratorHomographyNet模块都干了什么

In [1]:
import glob
import os
import cv2
import numpy as np
from dataGenerator import DataGeneratorHomographyNet

/home/nvidia/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
img_dir = os.path.join(os.path.expanduser("~"), "/home/nvidia/test2017")
img_ext = ".jpg"
img_paths = glob.glob(os.path.join(img_dir, '*' + img_ext))
dg = DataGeneratorHomographyNet(img_paths, input_dim=(240, 240))
data, label = dg.__getitem__(0)
for idx in range(dg.batch_size):
    cv2.imshow("orig", data[idx, :, :, 0])
    cv2.imshow("transformed", data[idx, :, :, 1])
    cv2.waitKey(0)

## 2、开始训练

In [5]:
import os
import glob
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import keras
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import tensorflow as tf
from homographyNet import HomographyNet
import dataGenerator as dg
keras.__version__

'2.1.2'

In [6]:
batch_size = 2
#取值0,1,2 0-安静模式 1-进度条 2-每一行都有输出
verbose = 1
#Epoch
nb_epo = 150
#计时开始
start_ts = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#用于训练的图片目录
data_path = "/home/nvidia/test2017"
#模型保存的目录
model_dir = "/home/nvidia"
img_dir = os.path.join(os.path.expanduser("~"), data_path)
model_dir = os.path.join(os.path.expanduser("~"), model_dir, start_ts)
#以时间为名创建目录
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

In [7]:
img_ext = ".jpg"
#获取所有图像目录
img_paths = glob.glob(os.path.join(img_dir, '*' + img_ext))
input_size = (360, 360, 2)
#划分训练集和验证集，验证集搞小一点，不然每个epoch跑完太慢了
train_idx, val_idx = train_test_split(img_paths, test_size=0.01)
#拿到训练数据
train_dg = dg.DataGeneratorHomographyNet(train_idx, input_dim=input_size[0:2], batch_size=batch_size)
#拿到既定事实的标签
val_dg = dg.DataGeneratorHomographyNet(val_idx, input_dim=input_size[0:2], batch_size=batch_size)
#对于神经网络来说这个鬼一样的图就是输入，它自己从这幅图的左边和右边学习出单应性矩阵，神奇吧？
#修正网络输入头
homo_net = HomographyNet(input_size)
#实例化网络结构
model = homo_net.build_model()
#输出模型
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 360, 360, 2)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 360, 360, 64)      1216      
_________________________________________________________________
batch_normalization_1 (Batch (None, 360, 360, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 360, 360, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 360, 360, 64)      36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 360, 360, 64)      256    

In [8]:
#检查点回调，没写tensorboard的回调，真正的大师都是直接看loss输出的
checkpoint = ModelCheckpoint(
    os.path.join(model_dir, 'model.h5'),
    monitor='val_loss',
    verbose=verbose,
    save_best_only=True,
    save_weights_only=False,
    mode='auto'
)

In [ ]:
#我嫌弃在上面改太麻烦，直接在这重定义了
#开始训练
#如果不加steps_per_epoch= 32,  就是每次全跑
history = model.fit_generator(train_dg, 
                              validation_data = val_dg,
                              #steps_per_epoch = 32, 
                              callbacks = [checkpoint], 
                              epochs = 15, 
                              verbose = 1)

Epoch 1/15
 1373/20131 [=>............................] - ETA: 1:18:50 - loss: 1615938396204833.0000 - mean_squared_error: 1615938396204833.0000

In [2]:
#整个图看看
history_df = pd.DataFrame(history.history)
history_df.to_csv(os.path.join(model_dir, 'history.csv'))
history_df[['loss', 'val_loss']].plot()
history_df[['mean_squared_error', 'val_mean_squared_error']].plot()
plt.show()

NameError: name 'history' is not defined

## 预测&评估

In [ ]:
TODO